In [ ]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)


In [1]:
# %load ../bin/data_walk.py
#!/usr/bin/env python
#Author: Duncan Tormey
#Email: dut@stowers.org or duncantormey@gmail.com
import pandas as pd
from glob import glob
from pprint import pprint


class LabData(object):

    def __init__(self, lab_path):
        self.lab_path = lab_path
        self.lab_name = [d for d in lab_path.split('/') if d][-1]
        self.lims_data = self.ret_lims_data()
        
    def __repr__(self):
        return self.lab_data.to_string()

    def __str__(self):
        return self.lab_data.to_string()

    def ret_sample_df(self, path):
        with open(path, 'r') as f:
            first_line = f.readline()
        columns = first_line.strip('\n').replace(' ', '_').split(',')
        sample_df = pd.read_csv(path, sep=',', skiprows=1, names=columns)
        sample_df['report_path'] = path
        sample_df['flowcell'] = path.split('/')[-2]

        return sample_df

    def ret_lims_data(self):
        wild_card = '%s/*/*/*/Sample_Report.csv' % self.lab_path
        sample_paths = glob(wild_card)
        sample_dfs = []
        for path in sample_paths:
            sample_df = self.ret_sample_df(path)
            sample_dfs.append(sample_df)
        lims_data = pd.concat(sample_dfs)
        lims_data.reset_index(inplace=True,drop=True)

        return lims_data
    


#if __name__ == "__main__":
lab_path = '/n/analysis/Baumann/'
baumann_lab = LabData(lab_path)

baumann_lab.lims_data.columns

Index(['#_reads', 'AlignPercent', 'IndexSequence1', 'IndexSequence2', 'Lab',
       'Lane', 'LibraryID', 'MeanReadLength', 'Order', 'OrderType', 'Output',
       'Read', 'ReadLength', 'Reference', 'SampleName', 'TotalReads', 'Type',
       'align_percent', 'custom_barcode', 'flowcell', 'illumina_index', 'lab',
       'lane', 'library_id', 'order', 'order_type', 'output',
       'pass_filter_percent', 'pass_filter_reads', 'read', 'read_length',
       'reference', 'report_path', 'sample_name', 'total_reads', 'type'],
      dtype='object')

In [2]:
baumann_lab.lims_data.lab.unique()

array(['Baumann Lab', nan, 'Conaway J Lab', 'Shilatifard Lab',
       'Trainor Lab', 'Zeitlinger Lab', 'Sanchez Lab', 'Si Lab',
       'Molecular Biology Facility', 'Li R Lab', 'Xie Lab', 'Li L Lab',
       'Workman Lab', 'Krumlauf Lab', 'Rohner Lab', 'Gerton Lab',
       'Gibson Lab', 'Cytometry Facility', 21916967.0, 23486434.0,
       24100260.0, 21278137.0, 23371835.0, 23394108.0, 24004940.0,
       16124298.0], dtype=object)

In [3]:
import subprocess
subprocess.call('md5sum ../data/test.txt > ../data/test.md5sum',shell=True)



1

In [244]:
#!/usr/bin/env python
# Author: Duncan Tormey
# Email: dut@stowers.org or duncantormey@gmail.com

import requests
import pandas as pd
from pandas.io.json import json_normalize 
import json
import os


order_search = """
    [{
        "type": "grouper",
        "groupType": "and",
        "items": [{
            "dataType": "stowers.department",
            "value": [%s],
            "comparison": "in",
            "id": "requestingDepartment",
            "entity": "MolecularBiology.Ngs.NgsRequest",
            "fieldId": "requestingDepartment"
        }, {
            "dataType": "selectlist",
            "value": ["COMPLETED"],
            "comparison": "in",
            "id": "status",
            "entity": "MolecularBiology.Ngs.NgsRequest",
            "fieldId": "status"
        }]
    }]
"""


sample_search = """
  [{
        "type": "grouper",
        "groupType": "and",
        "items": [{
            "dataType": "selectlist",
            "value": ["COMPLETED"],
            "comparison": "in",
            "id": "workRequest.status",
            "entity": "MolecularBiology.Ngs.NgsRequestSample",
            "fieldId": "workRequest.status"
        }, {
            "dataType": "stowers.department",
            "value": [%s],
            "comparison": "in",
            "id": "workRequest.requestingDepartment",
            "entity": "MolecularBiology.Ngs.NgsRequestSample",
            "fieldId": "workRequest.requestingDepartment"
        }, {
            "dataType": "textfield",
            "value": "%s",
            "comparison": "like",
            "id": "workRequest.prnOrderId",
            "entity": "MolecularBiology.Ngs.NgsRequestSample",
            "fieldId": "workRequest.prnOrderId"
        }]
    }]
"""


lib_search = """
    [{
        "type": "grouper",
        "groupType": "and",
        "items": [{
            "dataType": "stowers.department",
            "value": [%s],
            "comparison": "in",
            "id": "workRequest.requestingDepartment",
            "entity": "MolecularBiology.Ngs.Library",
            "fieldId": "workRequest.requestingDepartment"
        }, {
            "dataType": "textfield",
            "value": "%s",
            "comparison": "like",
            "id": "workRequest.prnOrderId",
            "entity": "MolecularBiology.Ngs.Library",
            "fieldId": "workRequest.prnOrderId"
        }]
    }]
""" 


def ret_lims_search_data(token_file, search, order=False): 
    if order:
        search = search % order
    
    with open(token_file, 'r') as fh:
        token = fh.readline().strip('\n')
    
    # URL for POST request
    url = 'https://lims.stowers.org/zanmodules/site/advanced-search/results.json'
    headers = {'Accept': 'application/json'}
    payload = {
        'search': search,
        'addGroups': ['request']
    }
    # IMPORTANT: API token must have "write" access to perform a POST
    request_results = requests.post(url=url, 
                                    auth=('apitoken', token), 
                                    data=json.dumps(payload), 
                                    headers=headers)

    json_data = request_results.json()
    
    return json_data


def ret_readable_order_data(json_order_data):
    df = json_normalize(json_order_data['results'])
    read_cols = [
        'analysisGoals',
        'comments',
        'completedAt',
        'coreCreatedLibraries',
        'createdAt',
        'creator.displayName',
        'creator.email',
        'creator.username',
        'displayString',
        'fulfiller.displayName',
        'fulfiller.email',
        'id',
        'machineMode.displayString',
        'machineMode.label',
        'machineType.displayString',
        'machineType.label',
        'ngsProjectType.displayString',
        'ngsProjectType.label',
        'numFlowcellsNeeded',
        'numLanesNeeded',
        'numPools',
        'numUnpooledLibraries',
        'orderTotal',
        'poolingComments',
        'prnOrderId',
        'readLength.displayString',
        'readLength.label',
        'readType',
        'requestTitle',
        'requestedAt',
        'requester.username',
        'requestingDepartment.departmentHead.displayName',
        'requestingDepartment.name',
        'specialInstructions',
        'staffComments',
        'updatedAt',
    ]
    
    df = df[read_cols]
    
    return df


def attchements_data_request(token_file, order_id):
    with open(token_file, 'r') as fh:
        token = fh.readline().strip('\n')
    url = 'https://lims.stowers.org/zanmodules/molecular-biology/ngs/requests/{}/attachments'.format(order_id)
    attachments_results = requests.get(
        url = url,
        auth=('apitoken', token),
    
    )
    
    attachments_json = attachments_results.json()
    attachments_df = json_normalize(attachments_json)
    
    return attachments_json, attachments_df


def download_attachments(token_file, attachments_df, dirpath=''):
    with open(token_file, 'r') as fh:
        token = fh.readline().strip('\n')
    outfiles = []
    if 'id' in attachments_df.columns:
        if len(attachments_df['id'].dropna()) > 0:
            for index, row in attachments_df.iterrows():
                url = 'https://lims.stowers.org/zanmodules/molecular-biology/ngs/requests/attachments/{}'.format(row['id'])
                attachment = requests.get(
                    url = url,
                    auth=('apitoken', token),
                )
                outfile = dirpath + '/' + row['name']
                with open(outfile, 'wb') as fw:
                    fw.write(attachment.content)
                outfiles.append(outfile)
    
    return outfiles

# Kit Items
# /zanmodules/molecular-biology/ngs/requests/{id}/kit-items
# or
# /zanmodules/molecular-biology/ngs/requests/{id}/unique-kit-items

# {id} here is the request ID like MOLNG-123

# Pools
# /zanmodules/molecular-biology/ngs/requests/{identifier}/pools

# {identifier} here is the request ID like MOLNG-123


def request_additional_oder_items(token_file, order_id, url_template):
    with open(token_file, 'r') as fh:
        token = fh.readline().strip('\n')
    url = url_template.format(order_id)
    results = requests.get(
        url = url,
        auth=('apitoken', token),
    
    )
    
    return results.json()


def save_json(obj, file_path):
    with open(file_path, 'w') as f:
        json.dump(obj, f, ensure_ascii=False)

        
def make_dir(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
        

#if __name__ == "__main__":
# constants
lims_department_id = "4"
token_file = '../data/lims_token.txt'

# constant modified searches
order_search = order_search % LIMS_DEPARTMENT_ID
sample_search = sample_search % (LIMS_DEPARTMENT_ID, '%s')
lib_search = lib_search % (LIMS_DEPARTMENT_ID, '%s')

###########################################################################
############################# Baumann Lab Run #############################
###########################################################################

#make root dir
root_meta_dir = '../data/lims_meta_data'
make_dir(root_meta_dir)

# get order data
order_data = ret_lims_search_data(token_file, order_search, order=False)
save_json(order_data, '%s/baumann_orders.json' % root_meta_dir)

order_data_df = ret_readable_order_data(order_data)
order_data_df.to_excel('%s/all_orders_meta_data.xls' % root_meta_dir, index=False)

# get order specific data
for index, row in order_data_df.iterrows():
    # assign order ids
    order_name = row['prnOrderId']
    order_id = row['id']
    
    # make order subdir
    order_sub_dir = root_meta_dir +'/%s_meta' % order_name
    make_dir(order_sub_dir)
    
    # get search data (library and samples)
    sample_data = ret_lims_search_data(token_file, sample_search, order=order_name)
    library_data = ret_lims_search_data(token_file, lib_search, order=order_name)
    save_json(sample_data, '%s/%s_sample_data.json' % (order_sub_dir, order_name))
    save_json(library_data, '%s/%s_library_data.json' % (order_sub_dir, order_name))
    
    # get attachments
    attachments_json, attachments_df = attchements_data_request(token_file, order_id)
    save_json(attachments_json, '%s/%s_attachments_data.json' % (order_sub_dir, order_name))
    download_attachments(token_file, attachments_df, dirpath=order_sub_dir)
    
    # get additional order items
    kit_url = 'https://lims.stowers.org/zanmodules/molecular-biology/ngs/requests/{}/kit-items'
    kit_items_data = request_additional_oder_items(token_file, order_name, url_template = kit_url)
    save_json(kit_items_data, '%s/%s_kit_items_data.json' % (order_sub_dir, order_name))
    
    pool_url = 'https://lims.stowers.org/zanmodules/molecular-biology/ngs/requests/{}/pools'
    pool_data = request_additional_oder_items(token_file, order_name, url_template = pool_url)
    save_json(pool_data, '%s/%s_pool_data.json' % (order_sub_dir, order_name))   
    
    
    

In [228]:
test_json, test_df = attchements_data_request(token_file, order_id=111854)
test_attachment_data = download_attachments(test_df, dirpath='')

test_attachment_data

['20141124_Atig001-003.tif',
 'MOLNG-1029_Labnotes.xlsx',
 'Summary of MOLNG-946 and MOLNG-1029.xlsx',
 'Repeat Project Notes.xlsx',
 'NOTES FOR FC 8-15.xlsx']

In [161]:
token_file = '../data/lims_token.txt'
json_order_data = ret_lims_data(token_file, order_search)
df = ret_readable_order_data(json_order_data)
df

,analysisGoals,comments,completedAt,coreCreatedLibraries,createdAt,creator.displayName,creator.email,creator.username,displayString,fulfiller.displayName,fulfiller.email,id,machineMode.displayString,machineMode.label,machineType.displayString,machineType.label,ngsProjectType.displayString,ngsProjectType.label,numFlowcellsNeeded,numLanesNeeded,numPools,numUnpooledLibraries,orderTotal,poolingComments,prnOrderId,readLength.displayString,readLength.label,readType,requestTitle,requestedAt,requester.username,requestingDepartment.departmentHead.displayName,requestingDepartment.name,specialInstructions,staffComments,updatedAt
0,Identify transcripts bound to RNA binding proteins,NaN,2018-03-08T13:13:44-06:00,True,2018-02-27T11:03:16-06:00,"Paez-Moscoso, Diego (djp)",djp@stowers.org,djp,MOLNG-2282,"Green, Troy (tmg)",tmg@stowers.org,127704,HiSeq-Rapid,HiSeq-Rapid,HiSeq,HiSeq,total RNA libraries (no selection),total RNA libraries (no selection),2.0,0,1,0,1946.00,Pool all together and run on two lanes or as suggested by MOLNG-2280. My samples should be pooled with MOLNG-2280.,MOLNG-2282,RapidSeq-50bp,RapidSeq-50bp,Single Read,NaN,2018-02-27T00:00:00-06:00,djp,"Baumann, Peter (peb)",Baumann Lab,Please process samples as MOLNG-2280 from Rachel Helston.,NaN,2018-03-08T13:13:44-06:00
1,identify RNAs bound by POP proteins,NaN,2018-03-08T13:13:44-06:00,True,2018-02-26T10:14:43-06:00,"Helston, Rachel (rmh)",rmh@stowers.org,rmh,MOLNG-2280: FLAG IPs,"Green, Troy (tmg)",tmg@stowers.org,127604,HiSeq-Rapid,HiSeq-Rapid,HiSeq,HiSeq,total RNA libraries (no selection),total RNA libraries (no selection),1.0,0,1,0,948.50,pool all together equally with libraries from DJP's upcoming order.,MOLNG-2280,RapidSeq-50bp,RapidSeq-50bp,Single Read,FLAG IPs,2018-02-26T00:00:00-06:00,rmh,"Baumann, Peter (peb)",Baumann Lab,NaN,NaN,2018-03-08T13:13:44-06:00
2,To assess RNA processing defects dependent upon presence of protein tags,NaN,2018-03-08T12:41:20-06:00,True,2018-02-22T17:25:37-06:00,"Helston, Rachel (rmh)",rmh@stowers.org,rmh,MOLNG-2277: Tagged POP proteins RNA-Seq,"Green, Troy (tmg)",tmg@stowers.org,127474,HiSeq-Rapid,HiSeq-Rapid,HiSeq,HiSeq,Ribo-dep Stranded RNA-Seq,Ribo-dep Stranded RNA-Seq,1.0,0,1,0,2779.35,pool all together equally,MOLNG-2277,RapidSeq-100bp,RapidSeq-100bp,Single Read,Tagged POP proteins RNA-Seq,2018-02-26T00:00:00-06:00,rmh,"Baumann, Peter (peb)",Baumann Lab,NaN,NaN,2018-03-08T12:41:20-06:00
3,NGS of A.neomexicana and A. tesselata to get primary assemblies to send to Dovetail Genomics.,NaN,2018-02-05T15:52:25-06:00,True,2018-01-19T10:26:22-06:00,"Schnittker, Robert (rrs)",rrs@stowers.org,rrs,MOLNG-2243: A.neomexican 16923 and A.teeselata 16537 NGS,"Egidy, Rhonda (rre)",rre@stowers.org,125653,HiSeq-Rapid,HiSeq-Rapid,HiSeq,HiSeq,DNA-Seq (1ug),DNA-Seq (1ug),2.0,0,2,0,7331.10,Please run these samples on separate flow cells across both lanes,MOLNG-2243,RapidSeq-250bp,RapidSeq-250bp,Paired Reads,A.neomexican 16923 and A.teeselata 16537 NGS,2018-01-19T00:00:00-06:00,rrs,"Baumann, Peter (peb)",Baumann Lab,NaN,NaN,2018-02-05T15:52:25-06:00
4,analyze hTR 3' end distribution,NaN,2017-12-27T14:45:28-06:00,False,2017-12-20T10:00:48-06:00,"Wang, Hui-Fang (hfw)",hfw@stowers.org,hfw,MOLNG-2232,NaN,NaN,124586,HiSeq-Rapid,HiSeq-Rapid,HiSeq,HiSeq,DNA-Seq (1ug),DNA-Seq (1ug),1.0,0,1,0,1696.50,Spike in MOLNG-2224 at 7%,MOLNG-2232,RapidSeq-250bp,RapidSeq-250bp,Paired Reads,NaN,2017-12-20T10:33:12-06:00,hfw,"Baumann, Peter (peb)",Baumann Lab,NaN,NaN,2017-12-27T14:45:28-06:00
5,Identify telomerase RNA modifications,NaN,2018-01-11T11:52:19-06:00,True,2017-12-19T10:25:02-06:00,"Fox, Cameron (ccf)",ccf@stowers.org,ccf,MOLNG-2230,"Hall, Kathryn (kmi)",kmi@stowers.org,124547,MinION,MinION,MinION,MinION,Oxford Nanopore - MinION,Oxford Nanopore - MinION,1.0,0,0,1,0.00,pre-pooled,MOLNG-2230,MinION-custom,MinION-custom,Single Read,NaN,2017-12-20T00:00:00-06:00,ccf,"Baumann, Peter (peb)",Baumann Lab,NaN,NaN,2018-01-11T11:52:19-06:00
6,NGS of A.tesselata lizard,NaN,2017-12-2

In [162]:
df[''].unique()

array([nan, '',
       '3/21: samples submitted (w/ 3 BA aliquots to run on smRNA chip - TapeStation results run by lab in Attachments).\n3/22: QC 3 samples designated by Rachel (79-81) with smRNA bioanalyzer assay\n3/22: lab asked us to wait for go ahead before continuing. \n3/23-24: library prep\n3/23: lab confirmed cycling conditions.\n3/24: QC libraries pre-size selection\n3/27: size selection and QC\n3/28: quantify and pool libraries ',
       '2/1/2017- Order submitted and samples received\n2/3/2017- Speed Vac\'d samples down to 5uL and began library construction at fragmentation\n2/6/2017- Adapter ligation and library QC\n2/7/2017- Library QC\n2/8/2017- PEB requested to not move forward with sequencing due to expected low abundance of RNA in experiment \n\nChanged order type to "total RNA libraries (no selection)" to ensure appropriate charges',
       "1/25/2017: (Wed) order submitted and samples rec'd.\n1/31: sample QC.\n2/1: sample shearing & QC.\n2/2: pre-library size select

In [154]:
uploadTargets = glob('/n/analysis/Baumann/*/MOLNG-*')
order_names = sorted([target.split('/')[-1] for target in  uploadTargets])


174 168
['/n/analysis/Baumann/ckt/MOLNG-1362', '/n/analysis/Baumann/ckt/MOLNG-1596', '/n/analysis/Baumann/ckt/MOLNG-1350', '/n/analysis/Baumann/ckt/MOLNG-1699', '/n/analysis/Baumann/ckt/MOLNG-1810', '/n/analysis/Baumann/ckt/MOLNG-1421', '/n/analysis/Baumann/ckt/MOLNG-1811', '/n/analysis/Baumann/ckt/MOLNG-1559', '/n/analysis/Baumann/ckt/MOLNG-1522', '/n/analysis/Baumann/ckt/MOLNG-1864', '/n/analysis/Baumann/ckt/MOLNG-1567', '/n/analysis/Baumann/ckt/MOLNG-1461', '/n/analysis/Baumann/ckt/MOLNG-1661', '/n/analysis/Baumann/ckt/MOLNG-1496', '/n/analysis/Baumann/klh/MOLNG-1271', '/n/analysis/Baumann/klh/MOLNG-1735', '/n/analysis/Baumann/klh/MOLNG-1928', '/n/analysis/Baumann/klh/MOLNG-2002', '/n/analysis/Baumann/klh/MOLNG-1204', '/n/analysis/Baumann/lak/MOLNG-1226', '/n/analysis/Baumann/lak/MOLNG-2144', '/n/analysis/Baumann/dut/MOLNG-1467', '/n/analysis/Baumann/lpn/MOLNG-1608', '/n/analysis/Baumann/lpn/MOLNG-182', '/n/analysis/Baumann/lpn/MOLNG-706', '/n/analysis/Baumann/lpn/MOLNG-1671', '/n/a

In [130]:
for x in df.columns:
    print('\'' + x + '\',')

'_objectType',
'analysisGoals',
'analysisResults',
'analysts',
'availableTransitions',
'billableServices',
'childCounts.indexedLibraries',
'childCounts.kitItems',
'childCounts.libraries',
'childCounts.pools',
'childCounts.samples',
'collaborationProjects',
'comments',
'completedAt',
'coreCreatedLibraries',
'createdAt',
'creator.displayName',
'creator.displayString',
'creator.email',
'creator.firstName',
'creator.jobTitle',
'creator.lastName',
'creator.loginName',
'creator.nickname',
'creator.phone',
'creator.photoUrl',
'creator.roles',
'creator.status',
'creator.uid',
'creator.username',
'defaultAccount.accountId',
'defaultAccount.award.awardNumber',
'defaultAccount.award.description',
'defaultAccount.award.id',
'defaultAccount.award.label',
'defaultAccount.award.monthlyLimit',
'defaultAccount.award.status',
'defaultAccount.displayString',
'defaultAccount.owningDepartment.displayString',
'defaultAccount.owningDepartment.id',
'defaultAccount.owningDepartment.name',
'defaultAccount.ownin

In [115]:
test_order = 'MOLNG-1029'
ret_lims_data(token_file, sample_search, order=test_order)

{'hitMaxPreVoterResults': False,
 'maximumQueryResults': 1000,
 'results': [{'canBeDeleted': False,
   'id': 11873,
   'isArchived': False,
   'isDeleted': False,
   'originalSample': {'canBeDeleted': True,
    'comments': '',
    'createdAt': '2017-04-05T21:10:15-05:00',
    'creator': {'displayName': 'Newton, Aracely (aan)',
     'displayString': 'Newton, Aracely (aan)',
     'email': 'aracelynewton@gmail.com',
     'firstName': 'Aracely',
     'jobTitle': 'Research Specialist I',
     'lastName': 'Newton',
     'loginName': 'aan',
     'phone': '816 926-4304',
     'photoUrl': 'https://lims.stowers.org/zanmodules/site/public/user/aan/photo',
     'roles': ['ROLE_USER', 'stowers.departmentManager'],
     'status': 'INACTIVE',
     'uid': 1708,
     'username': 'aan'},
    'erccSpikeIn': '',
    'experimentalVariables': [],
    'id': 11873,
    'isDeleted': False,
    'label': 'A_tigris8450',
    'owner': {'displayName': 'Newton, Aracely (aan)',
     'displayString': 'Newton, Aracely 

In [116]:
ret_lims_data(token_file, lib_search, order=test_order)

{'hitMaxPreVoterResults': False,
 'maximumQueryResults': 1000,
 'results': [{'_objectType': 'MolecularBiology.Ngs.Library',
   'adapterType': {'displayString': 'TruSeq Universal Adapter',
    'id': 742,
    'isActive': True,
    'isDeleted': False,
    'label': 'TruSeq Universal Adapter',
    'sort': 6},
   'canBeDeleted': False,
   'comments': 'Pippin size selected for 500-850bp',
   'constructedAt': '2015-03-05T00:00:00-06:00',
   'id': 11152,
   'indexBarcode': {'displayString': 'BiooSci-20',
    'id': 2024,
    'isActive': True,
    'isDeleted': False,
    'label': 'BiooSci-20',
    'sequence': 'GTGGCC',
    'sort': 19,
    'type': {'displayString': 'BiooScientific',
     'id': 896,
     'isActive': True,
     'isCustom': False,
     'isDeleted': False,
     'isDualIndex': False,
     'label': 'BiooScientific',
     'sort': 0}},
   'indexLabel': 'BiooScientific',
   'indexLocation': {'displayString': 'Illumina TruSeq Style',
    'id': 888,
    'isActive': True,
    'isDeleted': Fal

In [7]:
df2.columns

Index(['canBeDeleted', 'comments', 'createdAt', 'creator', 'erccSpikeIn',
       'experimentalVariables', 'id', 'isDeleted', 'label', 'owner',
       'primarySpecies', 'primarySpeciesAnalysisName', 'prnId', 'purity',
       'quantification', 'referenceUrls', 'speciesDisplayString',
       'startingMaterial', 'submittedConcentration', 'submittedVolume',
       'type'],
      dtype='object')